## Simple Mission in Guided-Mode

This notebook serves to get familiar with the simulator

In [1]:
import os

from config import (
    ARDU_LOGS_PATH,
    ARDUPILOT_VEHICLE_PATH,
    ENV_CMD_PYT,
    VEH_PARAMS_PATH,
    BasePort,
)
from helpers import clean, create_process, setup_logging, wait_for_port
from helpers.connections.mavlink.conn import create_tcp_conn
from helpers.coordinates import GRAPose
from planner import State
from planner.plan import Plan

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agree with first waypoint in mission.waypoint
gra_origin = GRAPose(lat=-35.3633245,lon=149.1652241,alt=0.0,heading=0)
spawn_str= gra_origin.to_str()

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
sysid = 1
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I{sysid-1} --sysid {sysid} "
    f"--no-rebuild "
    f"--use-dir={ARDU_LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f"--port-offset=0 "  # ArduPilot automatically binds TCP port 5760 (BasePort.ARP)
    f'--custom-location={spawn_str}'
)

create_process(
                vehicle_cmd,
                after="exec bash",
                visible= True,
                suppress_output=False,
                title="ardu_vehicle",
                env_cmd=ENV_CMD_PYT,
            )
wait_for_port(BasePort.ARP, timeout=1)

Waiting for port 5760 to open...


## Connect to the vehicle

In [3]:
conn = create_tcp_conn(
    base_port=BasePort.ARP, 
    offset=0,
    role="client", 
    src_sysid=sysid,
    src_compid=140,
    
)

print("✅ TCP connection established!")

✅ TCP connection established!


## Create Plan in guided mode

Action commands are sent online, in contrast to auto mode where the full plan is loaded into the UAV before starting the mission.

In [4]:
plan = Plan.square(side_len=5, alt=5, wp_margin=0.5, navegation_speed=5)

In [5]:
plan

🕓 <Plan '📋 Square Trajectory'>
  🕓 <Action '🛡️ PREARM'>
    🕓 <CheckDisarmed '🔹 Check disarmed'>
    🕓 <EKFStatus '🔹 Check EKF status'>
    🕓 <GPSStatus '🔹 Check GPS'>
    🕓 <CheckSystem '🔹 Check system status'>
  🕓 <Action '⚙️ MODE'>
    🕓 <SwitchMode '🔹 Switch to GUIDED'>
  🕓 <Action '🔒 ARM'>
    🕓 <Arm '🔹 arm'>
  🕓 <Action '🛫 TAKEOFF'>
    🕓 <TakeOff '🔹 take off to 1.0 m'>
  🕓 <Action '✈️ FLY'>
    🕓 <GoTo '🔹 go to ENU(x=0, y=0, z=5)'>
    🕓 <GoTo '🔹 go to ENU(x=0, y=5, z=5)'>
    🕓 <GoTo '🔹 go to ENU(x=5, y=5, z=5)'>
    🕓 <GoTo '🔹 go to ENU(x=5, y=0, z=5)'>
    🕓 <GoTo '🔹 go to ENU(x=0, y=0, z=5)'>
  🕓 <Action '🛬 LAND'>
    🕓 <Land '🔹 Land UAV'>

# Execute Plan in guided mode

In [6]:
plan.bind(conn,gra_origin.unpose())
setup_logging('plan',verbose=2)
while plan.state != State.DONE:
    plan.act()

15:02:48 - plan - DEBUG - ▶️ Vehicle 1: Plan Started: 📋 Square Trajectory
15:02:48 - plan - DEBUG - ▶️ Vehicle 1: Action Started: 🛡️ PREARM
15:02:48 - plan - DEBUG - ▶️ Vehicle 1: CheckDisarmed Started: Check disarmed
15:02:48 - plan - DEBUG - ✅ Vehicle 1: CheckDisarmed Done: Check disarmed
15:02:48 - plan - DEBUG - Vehicle 1: 📡 Requested message EKF_STATUS_REPORT at 1.00 Hz
15:02:48 - plan - DEBUG - ▶️ Vehicle 1: EKFStatus Started: Check EKF status
15:02:51 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
15:02:52 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
15:02:53 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
15:02:54 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
15:02:55 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_

In [7]:
print(plan)

✅ <Plan '📋 Square Trajectory'>
  ✅ <Action '🛡️ PREARM'>
    ✅ <CheckDisarmed '🔹 Check disarmed'>
    ✅ <EKFStatus '🔹 Check EKF status'>
    ✅ <GPSStatus '🔹 Check GPS'>
    ✅ <CheckSystem '🔹 Check system status'>
  ✅ <Action '⚙️ MODE'>
    ✅ <SwitchMode '🔹 Switch to GUIDED'>
  ✅ <Action '🔒 ARM'>
    ✅ <Arm '🔹 arm'>
  ✅ <Action '🛫 TAKEOFF'>
    ✅ <TakeOff '🔹 take off to 1.0 m'>
  ✅ <Action '✈️ FLY'>
    ✅ <GoTo '🔹 go to ENU(x=0, y=0, z=5)'>
    ✅ <GoTo '🔹 go to ENU(x=0, y=5, z=5)'>
    ✅ <GoTo '🔹 go to ENU(x=5, y=5, z=5)'>
    ✅ <GoTo '🔹 go to ENU(x=5, y=0, z=5)'>
    ✅ <GoTo '🔹 go to ENU(x=0, y=0, z=5)'>
  ✅ <Action '🛬 LAND'>
    ✅ <Land '🔹 Land UAV'>


In [8]:
plan.reset()
print(plan)

🕓 <Plan '📋 Square Trajectory'>
  🕓 <Action '🛡️ PREARM'>
    🕓 <CheckDisarmed '🔹 Check disarmed'>
    🕓 <EKFStatus '🔹 Check EKF status'>
    🕓 <GPSStatus '🔹 Check GPS'>
    🕓 <CheckSystem '🔹 Check system status'>
  🕓 <Action '⚙️ MODE'>
    🕓 <SwitchMode '🔹 Switch to GUIDED'>
  🕓 <Action '🔒 ARM'>
    🕓 <Arm '🔹 arm'>
  🕓 <Action '🛫 TAKEOFF'>
    🕓 <TakeOff '🔹 take off to 1.0 m'>
  🕓 <Action '✈️ FLY'>
    🕓 <GoTo '🔹 go to ENU(x=0, y=0, z=5)'>
    🕓 <GoTo '🔹 go to ENU(x=0, y=5, z=5)'>
    🕓 <GoTo '🔹 go to ENU(x=5, y=5, z=5)'>
    🕓 <GoTo '🔹 go to ENU(x=5, y=0, z=5)'>
    🕓 <GoTo '🔹 go to ENU(x=0, y=0, z=5)'>
  🕓 <Action '🛬 LAND'>
    🕓 <Land '🔹 Land UAV'>
